In [1]:
import praw
import pandas as pd

In [2]:
usrname = 'mpbot0'
cid = 'yUJmELsXRFcD6A'
csecret = 'VKquj58R_cCuv021SqH6DYkCUic'
psswd = 'abcd1234'

In [3]:
# create a Reddit instance
reddit = praw.Reddit(client_id=cid,
                     client_secret=csecret, password=psswd,
                     user_agent='aneesh', username=usrname)

In [4]:
path = r"C:\Users\anees\Documents\CS\MP\datasets\top200.csv"

In [5]:
slist = pd.read_csv(path, names=['subreddit'])
sublist_full = slist['subreddit']
print("No of subreddits: " + str(len(sublist_full)))
sublist = [x for x in sublist_full]

No of subreddits: 200


In [6]:
def getSubComments(comment, allComments, sub, verbose=False):
    c = comment.body
    if c not in ['[removed]','[deleted]'] and comment.author is not None:
        allComments.append([sub, comment.author.name, c, comment.score])
    if not hasattr(comment, "replies"):
        replies = comment.comments()
        if verbose: 
            print("fetching (" + str(len(allComments)) + " comments fetched total)")
    else:
        replies = comment.replies
    for child in replies:
        getSubComments(child, allComments, sub, verbose=verbose)

In [7]:
def getAll(submission, sub, verbose=False):
    comments = submission.comments
    submission.comments.replace_more(limit=10) # 0 for faster but less covering scrape
    commentsList = []
    for comment in comments:
        getSubComments(comment, commentsList,sub, verbose=verbose)
    return commentsList

In [12]:
full_comment = []
subreddit = reddit.subreddit('all')
top = subreddit.top('week',limit=200)
count = 0
non_top = 0
top_count = 0
exception_count = 0
post_count = 0
for submission in top:
    sub = submission.subreddit.display_name
    try:
        comments = getAll(submission, sub)
        if sub not in sublist:
            non_top+=1
        else:
            top_count+=1
        count+=len(comments)
        print(post_count,sub, submission.title, count)
        full_comment.extend(comments)      
    except praw.exceptions.PRAWException as e:
        exception_count+=1
        pass
    
    post_count+=1
    if count>=100000:
        break

0 movies Disney Reinstates Director James Gunn For ‘Guardians Of The Galaxy 3’ 1449
1 aww The decoy keyboard is working 2350
2 aww Everyone needs a friend like #50 3739
3 gifs Meanwhile in Hoboken New Jersey 4711
4 aww You are never too old to be a kid 5295
5 gifs p?xar 5958


KeyboardInterrupt: 

In [7]:
# full_comment = []
# subreddit = reddit.subreddit('all')
# top = subreddit.top('week',limit=200)
# count = 0
# non_top = 0
# removed = 0
# deleted = 0
# exception_count = 0
# post_count = 0
# for submission in top:
#     submission.comments.replace_more(limit=0)
#     sub = submission.subreddit.display_name
#     submission_comment_count = 0
#     for comment in submission.comments.list():

#         try:
#             if count%100==0:
#                 print(count, non_top)
#             if comment.score_hidden == True: # score is hidden due to being new thread
#                 break #thread is too new

#             if comment.body in ['[removed]','[deleted]']:
#                 if comment.body == '[removed]':
#                     removed+=1
#                 else:
#                     deleted+=1
#                 continue

#             c = comment.body
#             score = comment.score
#             full_comment.append([sub,c,score])
#             count+=1
#             if sub not in sublist:
#                 non_top+=1
#             submission_comment_count+=1
#         except praw.exceptions.PRAWException as e:
#             exception_count+=1
#             pass  

#         if submission_comment_count>1000:
#             print("moving to next submission")
#             break
#     post_count+=1
#     if count>=100000:
#         break

0 0
100 0
200 0
300 0
400 0
500 0
600 0
700 0
800 0
900 0
1000 0
1100 0
1200 0
1300 0
1400 0
1500 0
1600 0
1700 0
1800 0
1900 0
2000 0
2100 0
2200 0
2300 0
2400 0
2500 0
2600 0
2700 0
2800 0
2900 0


KeyboardInterrupt: 

In [11]:
df = pd.DataFrame(columns=['subreddit','author','comment','score'])

In [12]:
df.subreddit = [x[0] for x in full_comment]
df.author = [x[1] for x in full_comment]
df.comment = [x[2] for x in full_comment]
df.score = [x[3] for x in full_comment]

In [13]:
df.head()

,subreddit,comment,score
0,gifs,You can pinpoint the precise moment he is told...,12138
1,gifs,Don't know what the brand of pizza it is but i...,14046
2,gifs,That look off screen when someone must be tell...,19299
3,gifs,Haha! I was watching this kid at the hour rece...,2269
4,gifs,Can't wait till he's on Ellen and gets a lifet...,1798


In [ ]:
df.to_csv("top_week.csv", index=False)